<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #12: Análisis de sentimiento
`Fecha de entrega: Mayo 27, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [19]:
#Importar los mismos datos del taller 11

import pandas as pd 
import tweepy

keys = pd.read_csv("C:/Users/johnv/llaves.csv", header=None)
keys = dict(zip(keys[0],keys[1]))

auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)


tweets = tweepy.Cursor(api.search,
                       q=['Motos -RT'],
                       lang='es',
                       geocode='4.7110,-74.0721,100km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)

t = [tweet for tweet in tweets]


t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)
    
    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data


import numpy as np

from wordcloud import WordCloud, ImageColorGenerator

from nltk.corpus import stopwords
stopwords = stopwords.words("spanish")

import matplotlib.pyplot as plt
import PIL.Image
from IPython.display import display

import re

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    texto = " ".join(texto)
    return texto

data['pp'] = data.tweets.apply(lambda texto: pre_procesado(texto))

data.head()

,tweets,date,city,user_location,id,link,pp
0,"🔋🚴‍♂️Desde @SENA_Antioquia llega nuestro #podcast de #SENAAlAire con la historia de Leonardo Rendón, un emprendedor que le apostó a su proyecto de baterías de litio, que se pueden adaptar a bicicletas, patinetas, motos y vehículos eléctricos #FondoEmprender https://t.co/cE5IWZTibU",2021-05-25 00:05:00,Bogotá,"Bogotá, Colombia",1396980583138426883,https://twitter.com/i/web/stuatus/1396980583138426883,sena antioquia llega podcast senaalaire historia leonardo rendón emprendedor apostó proyecto baterías litio pueden adaptar bicicletas patinetas motos vehículos eléctricos fondoemprender https t co ce iwztibu
1,"@sarashortyyy @yisel_ma @melkorofangmar @ElPajaroGrand @MiveU00005 @GustavoBolivar Ah o sea su lógica es cambiar, sin importar que nos jodamos, es exguerrillero, y por lo tanto no se la pasaba en misa, tres días de basura? De que habla? Fue casi un mes, además se le olvida lo de las bolsas y lo de las motos eléctricas.",2021-05-24 23:46:19,Bogotá,"Bogotá, D.C., Colombia",1396975881936113672,https://twitter.com/i/web/stuatus/1396975881936113672,sarashortyyy yisel ma melkorofangmar elpajarogrand miveu gustavobolivar ah lógica cambiar importar jodamos exguerrillero pasaba misa tres días basura habla casi mes además olvida bolsas motos eléctricas
2,@ranoguera @manucalle87 @Eganbernal Lo tuvieron que haber grabado porque las motos siempre están en esas. Esperemos que la cuenta del Giro tenga la bondad de subir más material.,2021-05-24 23:12:44,Bogotá,"Bogotá, Colombia",1396967428689641480,https://twitter.com/i/web/stuatus/1396967428689641480,ranoguera manucalle eganbernal haber grabado motos siempre esperemos cuenta giro bondad subir material
3,"Estos Senadores votaron negativo, para no escuchar a las víctimas de la fuerza pública:\n\n1.Miguel Barreto\n2.Ernesto Macías\n3.Nicolas Araujo\n4.Amanda Rodríguez\n5.Carlos Mejia\n6.Carlos Motos\n7.María del Rosario\n8.Mila Romero\n9.Edgar Mizrachi\n\nNi un voto más, muerte política.",2021-05-24 22:12:37,Bogotá,Bogotá,1396952299944169472,https://twitter.com/i/web/stuatus/1396952299944169472,senadores votaron negativo escuchar víctimas fuerza pública miguel barreto ernesto macías nicolas araujo amanda rodríguez carlos mejia carlos motos maría rosario mila romero edgar mizrachi voto muerte política
4,"Y Benedettí y Roy barreras son muy impolutos?, la contratación a dedo en la alcaldía Petro es impoluta?, la corrupción en el acueducto en ese mandato, es impoluta?, el despilfarro en camiones dañados y motos de niños para ser usadas por la policía, es impoluto? https://t.co/Md3aa9RpN7",2021-05-24 21:53:49,Bogotá,Bogota-Colombia,1396947568484044802,https://twitter.com/i/web/stuatus/1396947568484044802,benedettí roy barreras impolutos contratación dedo alcaldía petro impoluta corrupción acueducto mandato impoluta despilfarro camiones dañados motos niños ser usadas policía impoluto https t co md aa rpn


###  `[25 pts]` Punto 1: Hacer análisis de sentimiento de los datos del anterior taller
- Columna con el puntaje numérico del sentimiento
- Pueden usar cualquiera de las librerias vistas en clase

In [20]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

vader = SentimentIntensityAnalyzer()
vader.polarity_scores(data['pp'])['compound']

data['sent_vader'] = data['pp'].apply(lambda val: vader.polarity_scores(val)['compound'])
data['sent_textblob'] = data['pp'].apply(lambda val: TextBlob(val).sentiment.polarity)
data[['pp', 'sent_vader', 'sent_textblob']].head()

data[['pp', 'sent_vader']].sort_values('sent_vader').tail()

from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load('sentiment')

def sent_flair(texto):
    frase = Sentence(texto)
    classifier.predict(frase)
    result = frase.labels[0]
    label = result.value
    score = result.score
    if label == 'NEGATIVE':
        return score * -1
    return score

data['sent_flair'] = data['pp'].apply(lambda val: sent_flair(val))
data.head()

2021-05-24 19:10:24,677 loading file C:\Users\johnv\.flair\models\sentiment-en-mix-distillbert_4.pt


,tweets,date,city,user_location,id,link,pp,sent_vader,sent_textblob,sent_flair
0,"🔋🚴‍♂️Desde @SENA_Antioquia llega nuestro #podcast de #SENAAlAire con la historia de Leonardo Rendón, un emprendedor que le apostó a su proyecto de baterías de litio, que se pueden adaptar a bicicletas, patinetas, motos y vehículos eléctricos #FondoEmprender https://t.co/cE5IWZTibU",2021-05-25 00:05:00,Bogotá,"Bogotá, Colombia",1396980583138426883,https://twitter.com/i/web/stuatus/1396980583138426883,sena antioquia llega podcast senaalaire historia leonardo rendón emprendedor apostó proyecto baterías litio pueden adaptar bicicletas patinetas motos vehículos eléctricos fondoemprender https t co ce iwztibu,0.0,0.0,0.997603
1,"@sarashortyyy @yisel_ma @melkorofangmar @ElPajaroGrand @MiveU00005 @GustavoBolivar Ah o sea su lógica es cambiar, sin importar que nos jodamos, es exguerrillero, y por lo tanto no se la pasaba en misa, tres días de basura? De que habla? Fue casi un mes, además se le olvida lo de las bolsas y lo de las motos eléctricas.",2021-05-24 23:46:19,Bogotá,"Bogotá, D.C., Colombia",1396975881936113672,https://twitter.com/i/web/stuatus/1396975881936113672,sarashortyyy yisel ma melkorofangmar elpajarogrand miveu gustavobolivar ah lógica cambiar importar jodamos exguerrillero pasaba misa tres días basura habla casi mes además olvida bolsas motos eléctricas,0.0,0.0,0.999607
2,@ranoguera @manucalle87 @Eganbernal Lo tuvieron que haber grabado porque las motos siempre están en esas. Esperemos que la cuenta del Giro tenga la bondad de subir más material.,2021-05-24 23:12:44,Bogotá,"Bogotá, Colombia",1396967428689641480,https://twitter.com/i/web/stuatus/1396967428689641480,ranoguera manucalle eganbernal haber grabado motos siempre esperemos cuenta giro bondad subir material,0.0,0.0,0.805779
3,"Estos Senadores votaron negativo, para no escuchar a las víctimas de la fuerza pública:\n\n1.Miguel Barreto\n2.Ernesto Macías\n3.Nicolas Araujo\n4.Amanda Rodríguez\n5.Carlos Mejia\n6.Carlos Motos\n7.María del Rosario\n8.Mila Romero\n9.Edgar Mizrachi\n\nNi un voto más, muerte política.",2021-05-24 22:12:37,Bogotá,Bogotá,1396952299944169472,https://twitter.com/i/web/stuatus/1396952299944169472,senadores votaron negativo escuchar víctimas fuerza pública miguel barreto ernesto macías nicolas araujo amanda rodríguez carlos mejia carlos motos maría rosario mila romero edgar mizrachi voto muerte política,0.0,0.0,-0.565767
4,"Y Benedettí y Roy barreras son muy impolutos?, la contratación a dedo en la alcaldía Petro es impoluta?, la corrupción en el acueducto en ese mandato, es impoluta?, el despilfarro en camiones dañados y motos de niños para ser usadas por la policía, es impoluto? https://t.co/Md3aa9RpN7",2021-05-24 21:53:49,Bogotá,Bogota-Colombia,1396947568484044802,https://twitter.com/i/web/stuatus/1396947568484044802,benedettí roy barreras impolutos contratación dedo alcaldía petro impoluta corrupción acueducto mandato impoluta despilfarro camiones dañados motos niños ser usadas policía impoluto https t co md aa rpn,0.0,0.0,-0.638769


###  `[25 pts]` Punto 2: ¿Cuál es el tuit más positivo y más negativo?

In [23]:
#Busqueda del Tweet mas positivo

data["sent_flair"].max()

0.9996862411499023

In [31]:
#Tweet mas positivo

data[data["sent_flair"] == data["sent_flair"].max()]

,tweets,date,city,user_location,id,link,pp,sent_vader,sent_textblob,sent_flair
82,"Ni se bajaron de las motos a ver si podían ver a la mujer en alguna parte... Es el momento en el que no se logra ver ni escuchar nada, solo espero que la mujer se encuentre bien y que no todas vivamos lo que ella vivió la noche de hoy. #mujer #ViolenciaDeGenero",2021-05-23 03:18:50,Bogotá,"Armenia, Colombia",1396304585829109760,https://twitter.com/i/web/stuatus/1396304585829109760,bajaron motos ver si podían ver mujer alguna parte momento logra ver escuchar solo espero mujer encuentre bien todas vivamos vivió noche hoy mujer violenciadegenero,0.0,0.0,0.999686


In [24]:
#Busqueda del tweet mas negativo

data["sent_flair"].min()

-0.9999022483825684

In [32]:
#Tweet mas negativo

data[data["sent_flair"] == data["sent_flair"].min()]

,tweets,date,city,user_location,id,link,pp,sent_vader,sent_textblob,sent_flair
23,"@camilochara En Calipso, en uno de los cierres, hacían bajar a la gente de las bicicletas y motos para llevarlas jaladas del timón. En glorieta de Siloé pude ver a una persona que inventó cruzar con su moto por un cierre y le sacaron cuchillo tamaño familiar para no seguir. Se devolvió.",2021-05-24 15:24:55,Bogotá,"Bogotá, Colombia",1396849699915812871,https://twitter.com/i/web/stuatus/1396849699915812871,camilochara calipso cierres hacían bajar gente bicicletas motos llevarlas jaladas timón glorieta siloé pude ver persona inventó cruzar moto cierre sacaron cuchillo tamaño familiar seguir devolvió,0.0,0.375,-0.999902
